# Visibility analysis

The visibility analysis will, at first, be conducted using the "Arealstatistik" (https://www.bfs.admin.ch/bfs/de/home/dienstleistungen/geostat/geodaten-bundesstatistik/boden-nutzung-bedeckung-eignung/arealstatistik-schweiz.assetdetail.25885691.html).
At a later stage it might be conducted using a DEM.

In [1]:
# setup
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import getpass

In [2]:
user_name = getpass.getuser()

# create a path to the CSV file
csv_file_path = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand', 'arealstatistik_Grindelwald.csv')

# load land use statistic data
arealstatistik_GW = pd.read_csv(csv_file_path, sep=';')
print(arealstatistik_GW)
arealstatistik_GW.info()

        E_COORD  N_COORD  AS18_4
0       2620000  1150100       2
1       2620000  1151400       4
2       2620000  1151800       4
3       2620000  1151900       4
4       2620000  1152200       4
...         ...      ...     ...
120696  2629900  1176900       3
120697  2630300  1177400       3
120698  2629000  1161300       4
120699  2629000  1162000       2
120700  2629000  1168000       4

[120701 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120701 entries, 0 to 120700
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   E_COORD  120701 non-null  int64
 1   N_COORD  120701 non-null  int64
 2   AS18_4   120701 non-null  int64
dtypes: int64(3)
memory usage: 2.8 MB


In [3]:
# create a path to the GeoJSON file of the roads
root = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand')
fn = 'Roads_small.geojson'
full_path = os.path.join(root, fn)
# load data
roads = gpd.read_file(full_path)
print(roads)
roads.info()

        fid  OBJECTID                                DKM_UUID  OBJEKTART  \
0        95      2126  {84F4C50E-E009-4DB8-AF3C-DEE20F3177F6}       1700   
1       113      2145  {6408AD08-C7B5-4602-81D7-A9B7FBFFD5FF}       1700   
2       114      2146  {578A6D84-CC7B-4687-B7C6-79313525392A}       1700   
3       163      2195  {BF0A1AEF-817C-45DA-A33C-6C37903D19BB}       1500   
4       169      2201  {C3EB4DD7-7E8F-4BB5-9DFD-BEE78567F8A7}       1400   
...     ...       ...                                     ...        ...   
5746  23006   1689149  {D0B1B4D9-0DE7-43E7-BE85-BAD1E985D1D4}       1500   
5747  23007   1689150  {55F1CCE5-76D5-4107-B685-4E3D3CA8F485}       1500   
5748  23008   1689151  {84EB5EDF-66A2-440D-A6C9-B53276A4295A}       1500   
5749  23018   1689161  {15A755D9-A618-4277-BB0E-B0D12769D6A6}       1700   
5750  23070   1689214  {A59DE7FF-AEF0-4373-904E-C7F76637246F}       1700   

      NAME ROUTENNUMMER   SG  STUFE  KUNSTBAUTE  BEFAHRBARKEIT  ...  \
0     None      

In [4]:
# create a path to the file of the DEM
dhm25_file_path = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand', 'DHM25_Grindelwald.geojson')
# load data
dhm25 = gpd.read_file(dhm25_file_path)
print(dhm25)

      OBJECTID                                    geometry
0      8651321  POINT Z (2625485.900 1172295.300 1584.000)
1      8651327  POINT Z (2627831.300 1172885.900 1505.000)
2      8652231  POINT Z (2639148.400 1170993.800 1431.000)
3      8652273  POINT Z (2648728.100 1170739.100 2928.000)
4      8652289  POINT Z (2652621.900 1170375.000 1713.000)
...        ...                                         ...
4275   8652221  POINT Z (2654487.500 1171484.400 1690.000)
4276   8653170   POINT Z (2659971.900 1171339.100 799.000)
4277   8683568  POINT Z (2625123.400 1157975.000 1292.000)
4278   8683716  POINT Z (2623037.500 1155435.900 1366.000)
4279   8683983  POINT Z (2632903.100 1152062.500 2773.000)

[4280 rows x 2 columns]


### Set-up

Here, classes are set up.

In [5]:
class Point():
    # initialise
    def __init__(self, x=None, y=None):
        self.x = x
        self.y = y

     # representation
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'


### Main (land use statistic)

Here the visibility analysis based on the land use statistic (Arealstatistik) is run.

In [6]:
x = 0
points_LUS = []

for _ in range(len(arealstatistik_GW)):
    a = (Point(arealstatistik_GW['E_COORD'].iloc[x], arealstatistik_GW['N_COORD'].iloc[x]))
    points_LUS.append(a)
    x += 1

print(f'The first 5 points are: {points_LUS[:5]}')
print(f'There are {len(points_LUS)} points in the dataset.')

The first 5 points are: [Point(x=2620000, y=1150100), Point(x=2620000, y=1151400), Point(x=2620000, y=1151800), Point(x=2620000, y=1151900), Point(x=2620000, y=1152200)]
There are 120701 points in the dataset.


In [7]:
# find closest pixel center for a given coordinate
def find_closest_pixel(x_coord, y_coord):
    pixel_centers = arealstatistik_GW[['E_COORD', 'N_COORD']].values
    
    distances = np.sqrt((pixel_centers[:, 0] - x_coord)**2 + (pixel_centers[:, 1] - y_coord)**2)
    
    closest_index = np.argmin(distances)
    
    closest_pixel_center = pixel_centers[closest_index]
    
    return closest_pixel_center

# test
x_coord = 2625321
y_coord = 1162324
closest_pixel_center = find_closest_pixel(x_coord, y_coord)
print("Closest pixel center:", closest_pixel_center)

Closest pixel center: [2625300 1162300]


In [8]:
# find neighbor of a given pixel center
def find_neighbors(center_pixel):
    x, y = center_pixel
    neighbors = []
    for dx in [-100, 0, 100]:
        for dy in [-100, 0, 100]:
            neighbors.append((x + dx, y + dy))
    return neighbors

# test
neighborhood = find_neighbors(closest_pixel_center)
print(neighborhood)

[(2625200, 1162200), (2625200, 1162300), (2625200, 1162400), (2625300, 1162200), (2625300, 1162300), (2625300, 1162400), (2625400, 1162200), (2625400, 1162300), (2625400, 1162400)]


In [9]:
# get landcover values for neighborhood
def get_values_for_pixels(pixel_centers):
    values = arealstatistik_GW.loc[(arealstatistik_GW['E_COORD'].isin([x for x, _ in pixel_centers])) & 
                                   (arealstatistik_GW['N_COORD'].isin([y for _, y in pixel_centers])), 'AS18_4']
    return values

# test
arealstatistik_values = get_values_for_pixels(neighborhood)
print(arealstatistik_values)

1236      3
62379     2
71885     2
82629     4
106077    3
110461    3
111485    4
113468    3
120482    4
Name: AS18_4, dtype: int64


In [10]:
# determine scenicness
def is_neighborhood_scenic(arealstatistik_values):
    count_1_3 = sum(1 for value in arealstatistik_values if value in [1, 3])

    if count_1_3 > 4:
        return False
    
    return True

# test
scenicness_test = is_neighborhood_scenic(arealstatistik_values)
print(scenicness_test)

True


## Main (viewshed)

Here the visibility analysis based on the DEM is run.

In [11]:
# split up the DHM25 geometry to fit in our own Point class
dhm25['E_COORD'] = dhm25['geometry'].x.round(1)
dhm25['N_COORD'] = dhm25['geometry'].y.round(1)
dhm25['Z_COORD'] = dhm25['geometry'].z.round(0).astype(int)
print(dhm25)

      OBJECTID                                    geometry    E_COORD  \
0      8651321  POINT Z (2625485.900 1172295.300 1584.000)  2625485.9   
1      8651327  POINT Z (2627831.300 1172885.900 1505.000)  2627831.3   
2      8652231  POINT Z (2639148.400 1170993.800 1431.000)  2639148.4   
3      8652273  POINT Z (2648728.100 1170739.100 2928.000)  2648728.1   
4      8652289  POINT Z (2652621.900 1170375.000 1713.000)  2652621.9   
...        ...                                         ...        ...   
4275   8652221  POINT Z (2654487.500 1171484.400 1690.000)  2654487.5   
4276   8653170   POINT Z (2659971.900 1171339.100 799.000)  2659971.9   
4277   8683568  POINT Z (2625123.400 1157975.000 1292.000)  2625123.4   
4278   8683716  POINT Z (2623037.500 1155435.900 1366.000)  2623037.5   
4279   8683983  POINT Z (2632903.100 1152062.500 2773.000)  2632903.1   

        N_COORD  Z_COORD  
0     1172295.3     1584  
1     1172885.9     1505  
2     1170993.8     1431  
3     1170739.1

In [12]:

# create points from this data
points_DHM25 = []

for i in range(len(dhm25)):
    point = Point(dhm25['E_COORD'].iloc[i], dhm25['N_COORD'].iloc[i])
    point.z = dhm25['Z_COORD'].iloc[i]  # Append 'Z_COORD' attribute to the point
    points_DHM25.append(point)

print(f'The first 5 points are: {points_DHM25[:5]}')
print(f'There are {len(points_DHM25)} points in the dataset.')
print(f'The elevation of the first point is {points_DHM25[0].z}m.a.s.l.')

The first 5 points are: [Point(x=2625485.9, y=1172295.3), Point(x=2627831.3, y=1172885.9), Point(x=2639148.4, y=1170993.8), Point(x=2648728.1, y=1170739.1), Point(x=2652621.9, y=1170375.0)]
There are 4280 points in the dataset.
The elevation of the first point is 1584m.a.s.l.
